# Day 3 - Conversational AI - aka Chatbot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [7]:
# Load the openai model from the github models
load_dotenv(override=True)
token = os.getenv("GITHUB_TOKEN")
endpoint = "https://models.inference.ai.azure.com"

model = "gpt-4o-mini"

openai = OpenAI(
    base_url=endpoint,
    api_key=token,
)

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [12]:
system_message = "You are a helpful assistant"

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)
    
    stream = openai.chat.completions.create(model=model, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:  # Check if choices exist and content is not None
            response += chunk.choices[0].delta.content
            yield response


## And then enter Gradio's magic!

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'I want to but a tie.'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'I want to but a tie.', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "When buying a tie, consider the following factors to ensure you choose the right one for your needs:\n\n1. **Occasion**: Are you buying a tie for a formal event, work, or casual wear? Different occasions may call for different styles.\n\n2. **Material**: Common materials for ties include silk, cotton, wool, and polyester. Silk is often preferred for dressy occasions because of its sheen and feel.\n\n3. **Color and Pattern**: Think about what colors and patterns you would like. Solid colors are versatile, while patterns like stripes, polka dots, or paisleys can add personality.\n\n4. **Width**: Tie widths vary. Wider ties (typically 3-4 inches) are more traditional, while skinny ties (around

In [15]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    
    stream = openai.chat.completions.create(model=model, messages=messages, stream=True)
    
    response = ""
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:  # Check if choices exist and content is not None
            response += chunk.choices[0].delta.content
            yield response

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


In [18]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


In [22]:
def chat(message, history):
    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]
    
    stream = openai.chat.completions.create(model=model, messages=messages, stream=True)
    
    response = ""
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:  # Check if choices exist and content is not None
            response += chunk.choices[0].delta.content
            yield response

In [23]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.
